In [1]:
import numpy as np
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
import pandas as pd
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Convolution1D, Dropout,MaxPooling1D
from keras.optimizers import SGD
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_excel('completeEncodedAndScaledDataset.xlsx')
df.head()


,Protocol_type,Service,Flag,Src_bytes,Logged_in,Count,Srv_count,Serror_rate,Srv_serror_rate,Rerror_rate,...,Dst_host_srv_count,Dst_host_same_srv_rate,Dst_host_diff_srv_rate,Dst_host_same_src_port_rate,Dst_host_srv_diff_host_rate,Dst_host_serror_rate,Dst_host_srv_serror_rate,Dst_host_rerror_rate,attack_type,attack_type_encoded
0,2.038571,0.629977,0.735369,-0.011190,-0.732914,-0.581217,-0.367299,-0.556584,-0.552030,-0.460806,...,-1.050270,-1.193023,2.187298,2.137976,-0.263701,-0.561390,-0.549814,-0.487765,normal,16
1,-0.154786,0.938904,-0.660992,-0.011262,-0.732914,0.275339,-0.312117,1.851192,1.851769,-0.460806,...,-0.821669,-0.966271,-0.237144,-0.498320,-0.263701,1.904034,1.899674,-0.487765,neptune,14
2,-0.154786,-0.543948,0.735369,-0.011147,1.364417,-0.643512,-0.323153,-0.075029,-0.071270,-0.460806,...,1.272317,1.074493,-0.457548,-0.408446,0.094049,-0.487427,-0.525319,-0.487765,normal,16
3,-0.154786,-0.543948,0.735369,-0.011163,1.364417,-0.448840,-0.025167,-0.556584,-0.552030,-0.460806,...,1.272317,1.074493,-0.457548,-0.498320,-0.263701,-0.561390,-0.549814,-0.487765,normal,16
4,-0.154786,0.938904,-2.057353,-0.011262,-0.732914,0.259766,-0.168642,-0.556584,-0.552030,2.274941,...,-0.885678,-1.034297,-0.148983,-0.498320,-0.263701,-0.561390,-0.549814,2.476323,neptune,14


In [3]:
df=df.drop(columns=['attack_type'])
df

,Protocol_type,Service,Flag,Src_bytes,Logged_in,Count,Srv_count,Serror_rate,Srv_serror_rate,Rerror_rate,...,Dst_host_count,Dst_host_srv_count,Dst_host_same_srv_rate,Dst_host_diff_srv_rate,Dst_host_same_src_port_rate,Dst_host_srv_diff_host_rate,Dst_host_serror_rate,Dst_host_srv_serror_rate,Dst_host_rerror_rate,attack_type_encoded
0,2.038571,0.629977,0.735369,-0.011190,-0.732914,-0.581217,-0.367299,-0.556584,-0.552030,-0.460806,...,0.656445,-1.050270,-1.193023,2.187298,2.137976,-0.263701,-0.561390,-0.549814,-0.487765,16
1,-0.154786,0.938904,-0.660992,-0.011262,-0.732914,0.275339,-0.312117,1.851192,1.851769,-0.460806,...,0.656445,-0.821669,-0.966271,-0.237144,-0.498320,-0.263701,1.904034,1.899674,-0.487765,14
2,-0.154786,-0.543948,0.735369,-0.011147,1.364417,-0.643512,-0.323153,-0.075029,-0.071270,-0.460806,...,-1.709884,1.272317,1.074493,-0.457548,-0.408446,0.094049,-0.487427,-0.525319,-0.487765,16
3,-0.154786,-0.543948,0.735369,-0.011163,1.364417,-0.448840,-0.025167,-0.556584,-0.552030,-0.460806,...,0.656445,1.272317,1.074493,-0.457548,-0.498320,-0.263701,-0.561390,-0.549814,-0.487765,16
4,-0.154786,0.938904,-2.057353,-0.011262,-0.732914,0.259766,-0.168642,-0.556584,-0.552030,2.274941,...,0.656445,-0.885678,-1.034297,-0.148983,-0.498320,-0.263701,-0.561390,-0.549814,2.476323,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37036,2.038571,-1.223588,0.735369,-0.011241,-0.732914,0.532306,2.226285,-0.556584,-0.552030,-0.460806,...,0.656445,1.272317,1.074493,-0.457548,-0.468362,-0.263701,-0.561390,-0.549814,-0.487765,16
37037,-0.154786,-0.543948,0.735369,-0.011096,1.364417,-0.612365,-0.267971,-0.556584,-0.552030,-0.460806,...,0.656445,1.080292,0.893092,-0.369387,-0.498320,-0.263701,-0.561390,-0.549814,-0.339561,16
37038,-0.154786,1.618544,-1.708262,-0.011262,-0.732914,-0.674660,-0.301080,-0.556584,-0.215498,2.274941,...,-0.163882,-0.181586,-0.354042,-0.325306,-0.468362,-0.084826,-0.438119,-0.353855,2.031710,11
37039,-0.154786,1.433188,-2.057353,-0.011262,-0.732914,-0.643512,-0.323153,-0.556584,-0.552030,1.727792,...,-0.226984,-0.583924,-0.762195,-0.325306,-0.468362,0.094049,-0.561390,-0.549814,2.120633,11


In [4]:
# split into input (X) and output (Y) variables
X = df.drop(columns=['attack_type_encoded'])
Y = df[['attack_type_encoded']]
print(X)
print(Y)




       Protocol_type   Service      Flag  Src_bytes  Logged_in     Count  \
0           2.038571  0.629977  0.735369  -0.011190  -0.732914 -0.581217   
1          -0.154786  0.938904 -0.660992  -0.011262  -0.732914  0.275339   
2          -0.154786 -0.543948  0.735369  -0.011147   1.364417 -0.643512   
3          -0.154786 -0.543948  0.735369  -0.011163   1.364417 -0.448840   
4          -0.154786  0.938904 -2.057353  -0.011262  -0.732914  0.259766   
...              ...       ...       ...        ...        ...       ...   
37036       2.038571 -1.223588  0.735369  -0.011241  -0.732914  0.532306   
37037      -0.154786 -0.543948  0.735369  -0.011096   1.364417 -0.612365   
37038      -0.154786  1.618544 -1.708262  -0.011262  -0.732914 -0.674660   
37039      -0.154786  1.433188 -2.057353  -0.011262  -0.732914 -0.643512   
37040       2.038571  0.938904  0.735369  -0.011242  -0.732914 -0.651299   

       Srv_count  Serror_rate  Srv_serror_rate  Rerror_rate  ...  \
0      -0.367299   

In [5]:
nb_features=21 #input parameters
nb_class=40 #output classes


In [6]:
Y = to_categorical(Y, nb_class)

In [7]:
print(X.shape)
print(Y.shape)


(37041, 21)
(37041, 40)


In [8]:
# reshape data
X = np.zeros((len(X), 1, nb_features))
print(X.shape)

(37041, 1, 21)


In [9]:
Y.shape

(37041, 40)

In [10]:
trainx, testx, trainy, testy = train_test_split(X,Y, test_size=0.25, random_state=42)

In [11]:
print(trainx.shape)
print(trainy.shape)
print(testx.shape)
print(testy.shape)

(27780, 1, 21)
(27780, 40)
(9261, 1, 21)
(9261, 40)


In [46]:
model = Sequential()
model.add(Convolution1D(filters=64, kernel_size=1, input_shape=(1,21)))
model.add(Convolution1D(filters=64, kernel_size=1))
#model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dropout(0.1))
model.add(Dense(100, activation='relu'))
model.add(Dense(nb_class,activation="sigmoid"))
model.summary()

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_31 (Conv1D)          (None, 1, 64)             1408      
                                                                 
 conv1d_32 (Conv1D)          (None, 1, 64)             4160      
                                                                 
 flatten_12 (Flatten)        (None, 64)                0         
                                                                 
 dropout_12 (Dropout)        (None, 64)                0         
                                                                 
 dense_24 (Dense)            (None, 100)               6500      
                                                                 
 dense_25 (Dense)            (None, 40)                4040      
                                                                 
Total params: 16,108
Trainable params: 16,108
Non-tra

In [47]:
sgd = SGD(lr=0.01, nesterov=True, decay=1e-6, momentum=0.9)
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

model.fit(trainx, trainy, epochs=25, validation_data=(testx, testy),batch_size=64)


Epoch 1/25
435/435 [==============================] - 2s 3ms/step - loss: 3.4296 - accuracy: 0.4181 - val_loss: 3.1847 - val_accuracy: 0.4173
Epoch 2/25
435/435 [==============================] - 1s 3ms/step - loss: 2.9692 - accuracy: 0.4224 - val_loss: 2.7745 - val_accuracy: 0.4173
Epoch 3/25
435/435 [==============================] - 1s 2ms/step - loss: 2.6074 - accuracy: 0.4224 - val_loss: 2.4687 - val_accuracy: 0.4173
Epoch 4/25
435/435 [==============================] - 1s 2ms/step - loss: 2.3525 - accuracy: 0.4224 - val_loss: 2.2674 - val_accuracy: 0.4173
Epoch 5/25
435/435 [==============================] - 1s 2ms/step - loss: 2.1924 - accuracy: 0.4224 - val_loss: 2.1476 - val_accuracy: 0.4173
Epoch 6/25
435/435 [==============================] - 1s 2ms/step - loss: 2.0990 - accuracy: 0.4224 - val_loss: 2.0794 - val_accuracy: 0.4173
Epoch 7/25
435/435 [==============================] - 1s 2ms/step - loss: 2.0461 - accuracy: 0.4224 - val_loss: 2.0408 - val_accuracy: 0.4173
Epoch 

In [48]:
model.save('1dmodel.h5')

In [49]:
import tensorflow

In [51]:
result=['apache2','back', 'buffer_overflow', 'ftp_write', 'guess_passwd',
 'httptunnel', 'imap', 'ipsweep', 'land', 'loadmodule', 'mailbomb', 'mscan',
 'multihop', 'named', 'neptune', 'nmap', 'normal', 'perl', 'phf', 'pod',
 'portsweep', 'processtable', 'ps', 'rootkit', 'saint', 'satan', 'sendmail',
 'smurf', 'snmpgetattack', 'snmpguess', 'spy', 'sqlattack', 'teardrop',
 'udpstorm', 'warezclient', 'warezmaster', 'worm', 'xlock', 'xsnoop', 'xterm']


In [52]:
model =tensorflow.keras.models.load_model('1dmodel.h5')

test_data = np.array([[-0.15478617,1.61854439,  0.73536923, -0.0112619 ,-0.54, -0.73291357, -0.6746598,-0.2458976 , -0.55658419, -0.35972625 ,-0.4608058,   0.72192361 ,-0.3773118,-0.1323311 , -0.8948217,  -1.17034772, -0.36938667, -0.46836203 , -0.56138989, -0.54981386 ,-0.48776502]])
o=model.predict(test_data.reshape(-1,1,21), batch_size=1)

print(o)
print(len(o[0]))
print(o.argmax())
print(result[int(o.argmax())])

1/1 [==============================] - 0s 61ms/step
[[0.3887703  0.28424335 0.01612915 0.01669627 0.70079124 0.14949572
  0.01263991 0.4612645  0.01152894 0.0084822  0.18762247 0.5064953
  0.04032083 0.01681391 0.921407   0.28398988 0.9529005  0.00600629
  0.01678917 0.06312399 0.47750074 0.47988686 0.01538797 0.02507758
  0.23688164 0.39999244 0.03826429 0.60100806 0.12827802 0.22249548
  0.00645022 0.00778927 0.1800741  0.01199665 0.20219372 0.48783812
  0.01204267 0.01366032 0.00799537 0.0146735 ]]
40
16
normal
